In [3]:
import gstlearn as gl
import numpy as np

# Creation of the class

In [19]:
class LocalKriging :
    def __init__(self,driftFunctions,db,model):        
        dbc = gl.Db(db)
        namesCoords = db.getNamesByLocator(gl.ELoc.X)
        coords = db[namesCoords]  
        modelc = gl.Model(model)
        driftsSymbol=[]
        for i,f in enumerate(driftFunctions):
            dbc["addedDrift" + str(i)] = f(coords)
            driftsSymbol += ["f" + str(i+1)]
        
        modelc.setDriftIRF(0,len(driftFunctions))
        
        dbc.setLocators(["addedDrift*"],gl.ELoc.F)
        self.drifts = driftFunctions
        #self.coeffs = np.array(krigingMethodDual(db,modelc)) # Dual
        self.covanisolist = model.getCovAnisoList()
        self.coordsData = []
        v = gl.VectorDouble(coords.shape[1])
        for i in range(coords.shape[0]):
            for j in range(coords.shape[1]):
                v[j] = coords[i,j]
            self.coordsData+= [gl.SpacePoint(v)]
            
    def eval(self,coordsTarget):
        c0 = np.zeros(shape= len(self.coordsData) + len(self.drifts)+1)
        coordsV = gl.VectorDouble(len(coordsTarget))
        for j in range(len(coordsTarget)):
            coordsV[j] = coordsTarget[j]
        for i in range(len(self.coordsData)):
            c0[i] = self.covanisolist.eval(0,0,self.coordsData[i],gl.SpacePoint(coordsV))
        c0[len(self.coordsData)] = 1.
        for i in range(len(self.drifts)):
            c0[1+i+len(self.coordsData)] = self.drifts[i](np.atleast_2d(np.array(coordsTarget)))
        return(c0)
#return np.sum(c0 * self.coeffs)

## Db creation

In [20]:
db = gl.Db.create()
db["x"] = np.random.uniform(size=100)
db["y"] = np.random.uniform(size=100)
db["z"] = np.random.normal(size=100)
db.setLocators(["x","y"],gl.ELoc.X)

## Drift functions 

In [21]:
def driftFunction1(coords):
    return coords[:,0]**2

def driftFunction2(coords):
    return coords[:,1]**2

driftFunctions = [driftFunction1,driftFunction2]

## Model

In [22]:
model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=1)


## Class instanciation

In [23]:
A = LocalKriging(driftFunctions,db,model)
A.eval(np.array([0.3,0.4]))

array([0.47847288, 0.56057999, 0.30098041, 0.59724798, 0.53258686,
       0.62390831, 0.51335152, 0.24449803, 0.3111087 , 0.46449411,
       0.12493232, 0.2574031 , 0.14064844, 0.30616229, 0.4236204 ,
       0.47168519, 0.86514077, 0.2484456 , 0.7592505 , 0.30719522,
       0.7434436 , 0.6048631 , 0.42783176, 0.14246811, 0.20074028,
       0.56859717, 0.80761694, 0.51965338, 0.24719912, 0.27122862,
       0.38707961, 0.38315159, 0.53956798, 0.76172463, 0.29065107,
       0.15704116, 0.28169211, 0.81980144, 0.26502887, 0.21498394,
       0.92501593, 0.20572603, 0.35628322, 0.52409144, 0.23926319,
       0.59849412, 0.25107677, 0.66161538, 0.21559167, 0.11993447,
       0.37547932, 0.29679441, 0.40745798, 0.27721619, 0.7023021 ,
       0.94689798, 0.20825125, 0.75360706, 0.51398962, 0.59376743,
       0.21518886, 0.38047622, 0.44511218, 0.72344995, 0.17267829,
       0.48547803, 0.1573785 , 0.19872399, 0.21553671, 0.17353159,
       0.47277913, 0.47239539, 0.52968152, 0.31278817, 0.74031